# Neural Networks Model using Keras and TensorFlow

## Objectives:
-Create a neural network model with Keras

-Evaluate models using cross-validation with scikit-learn 

-Tune model with Keras

## Dataset Visualization:

In [29]:
import numpy
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from scipy.stats import zscore
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
import pandas as pd

# load dataset 
dataframe = pd.read_csv("DataNVMpy.csv") 
dataset = dataframe.values

In [26]:
dataframe.head() #verifying that we could read the data correctly

,price,size,canalhouse,rooms,balconyroofterrace,parking,carport,garage,garden,maintenancegood,centralheating,insulation,listed
0,1305000,500,0,15,0,1,0,1,0,0,1,1,0
1,1250000,500,0,15,0,1,0,1,0,0,1,1,0
2,1225000,500,0,5,0,1,0,1,0,1,1,1,0
3,3150000,475,0,10,0,1,0,1,0,1,1,1,0
4,2050000,475,0,5,1,1,1,0,1,1,1,0,0


## Baseline Neural Network Model 

In [27]:



#split data into X and Y variables:
X = dataset[:,1:13]
Y = dataset[:,0]

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.20, random_state=42)

# define base model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=12, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# evaluate model
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))


Baseline: -20829310931.05 (24776257816.41) MSE


In [19]:
print(X)


[[500   0  15 ...   1   1   0]
 [500   0  15 ...   1   1   0]
 [500   0   5 ...   1   1   0]
 ...
 [ 26   0   1 ...   1   1   0]
 [ 26   0   1 ...   1   1   0]
 [ 26   0   1 ...   1   1   0]]


In [20]:
print(Y)


[1305000 1250000 1225000 ...  135000   88000   87000]


In [28]:
model = Sequential()
model.add(Dense(12, input_dim=12, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
checkpointer = ModelCheckpoint(filepath="best_weights.hdf5", verbose=0, save_best_only=True) # save best model
model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpointer],verbose=0,epochs=1000)
model.load_weights('best_weights.hdf5') # load weights from best model

# Predict and measure RMSE
pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Score (RMSE): {}".format(score))

# Plot the chart
chart_regression(pred.flatten(),y_test)

NameError: name 'EarlyStopping' is not defined